# Migrating to Pydantic

It makes sense to migrate to a well supported package for handling metadata, like `Pydantic`.  

This will also remove the costly step of reading the JSON files each time an object is initiated and weird things that happen when you set `_attr_dict`.  

Steps:

1. Convert existing JSON to JSON Schema files and move to `mt_metadata.mt_metadata.standards` folder for reference later.
2. Convert the JSON Schema files to `pydantic.BaseModel` objects using `datamodel-code-generator`

## 1. Convert existing JSON

This is relatively straight forward just a simple mapping and inclusion of some other things.  The mapping is being done in `mt_metadata.utils.converters.to_json_schema`.

## 2. Convert JSON Schema to pydantic.BaseModel

This is tougher.  For now use `datamodel-code-generator`.

```
datamodel-codegen.exe --input .\person_schema.json --input-file-type jsonschema --field-constraints --formatters black --use-annotated --output person_model.py
```

In [1]:
%load_ext autoreload
%autoreload 2

In [ ]:
from typing import Literal, Optional, Annotated, List
from pydantic import BaseModel, Field, ConfigDict, create_model
from pydantic.fields import PrivateAttr, FieldInfo
import json

from mt_metadata.utils.mttime import MTime



In [2]:
def custom_docstring():
    def decorator(cls):
        cls.__doc__ = cls.model_json_schema()["properties"]
        return cls
    return decorator

# @custom_docstring()
# class Order(BaseModel):
#     order_id: int
#     amount: float

# print(Order.__doc__)

In [ ]:
class DocPreservingModelMetaclass(ModelMetaclass):
     def __new__(mcs, name, bases, namespace, **kwargs):
         doc = namespace.pop('__doc__', None)
         cls = super().__new__(mcs, name, bases, namespace, **kwargs)
         cls.__doc__ = doc
         return cls

In [1]:
from mt_metadata.common.comment_basemodel import Comment
from mt_metadata.timeseries.declination_basemodel import Declination
from mt_metadata.common.location_basemodel import Location


In [2]:
l = Location(latitude=10, longitude=20, elevation=30)

In [3]:
l

{
    "location": {
        "datum": "WGS84",
        "elevation": 0.0,
        "latitude": 0.0,
        "longitude": 0.0
    }
}

In [ ]:
d = Declination()
d.comments 

In [4]:
d = f.to_dict()


In [2]:
from mt_metadata.utils import units

In [3]:
u = units.get_unit_object("km per s ohm")

In [6]:
from mt_metadata.timeseries.filtered_basemodel import Filtered, AppliedFilter

In [7]:
f = Filtered(filter_list=[AppliedFilter(name="test", filter_type="lowpass", cutoff=0.1, order=2)])

In [8]:
a = Filtered()
a.from_dict(f.to_dict())